# Setting up Paths

In [1]:
#path to the data
pathCOMPASOutput = '/home/cneijssel/Documents/Projects/Data/CosmicInt/'
#path to compasrepo which contains ther popsynth folder
pathRepo = '/home/cneijssel/Documents/COMPASpop'
#path to Bootstrap
pathBootstrap = '/home/cneijssel/Documents/Projects/Data/CosmicInt/'


# Imports

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import h5py as h5
import time
import os
import sys
import copy
#Quick fudge to make import from ../Scripts work
import sys
sys.path.append(pathRepo + '/popsynth/Papers/NeijsselEtAL/CosmicIntegration/Scripts')


# Custom Imports from COMPASrepo

In [3]:
import ClassCosmicIntegrator  as CI #Given settings and redshifts returns rates (2D arrays) Loads the data
import ClassBootStrap         as CB #pre sampled from initial seeds per metallicity the reader is there too
import coencodeVarious        as CV
import ClassBayes             as CL  #Used to create KDECOMPAS/Posterior/Likelyhood 
import ClassEvents            as CE  #Where I store the GW-events observed


# General Idea

In the previous notebook we cerated an h5 file with the rate per chirp mass per system per variation. In this notebook we will calculate the likelyhood per event and the total likelyhood. Note that for each individual likely hood, whether it is the sum or not, we need to bootstrap individually.

In [4]:
h5file           = h5.File('VariationIntegration.h5')

# Results        a 2D array of 18 variations 2X3X3 (rows) and 
#                              5 types of rates 2X3 (columns) and + - error so 15 columns
results          = np.zeros(shape=(18,15))

#keys are not given sorted, however in the sorted order it is easier to make the table
#convert list of string numbers to integers, get the arguments to sort and use these
#to sort keys
a = np.array(h5file.keys())
b = np.argsort(a.astype(int))

#for v in a[b]:
    #print h5file[v]['SFR_MZ_GSMF'][...]
#    print h5file[v]['BBH'].keys()

# Bootstrap and Events

I have premade a bootsStrap wit ClassBootstrap, this is a resampling off all seeds per metallicity with replacement. From that I count of *all* the DCOs in my double compact object file how often they appear. 
That is stored as a column to file. Hence if I have a maskDCO of one file say cosmic integration, that same maskDCO applied to the bootstrap file gives me the nr of draws. Hence here I only have to recover the mask, and apply to bootstrap file to get counts. Then I sort the counts and get +- 90% uncertainty


ClassEvents contains the information (as of yet) fo the events


In [5]:
boot   = CB.Bootstrap(pathToFile = pathBootstrap, fileName='Bootstrap5.h5')

def bootstrap(Likelihoods, cene):
    #order them from low to high and give error 
    Rates     = np.sort(Likelihoods) - np.sum(ratePerSystem)
    #If you sort array at what index do you find lower and upper confidence 90%
    lower     = int((nrSamples/100.)*5)
    upper     = int((nrSamples/100.)*95)
    return Rates[lower], Rates[upper]






events = CE.Observations()    

names, values, lowerErrors, upperErrors, surveys =\
events.giveParameterAndError(parameter='Mchirp', types=['BBH'])
print names, surveys


['GW150914' 'GW151012' 'GW151226' 'GW170104' 'GW170608' 'GW170729'
 'GW170809' 'GW170814' 'GW170818' 'GW170823'] ['O1' 'O1' 'O1' 'O2' 'O2' 'O2' 'O2' 'O2' 'O2' 'O2']


In [6]:
def Likelihoods(chirpMasses, rates, maskDCO):
    

    nrSamples = 200

    
    #calculate the width of the kernel using scipy in Bayes class
    bayes = CL.BayesianLikelyhoodSingleObservation(datapoints=chirpMasses)
    
    
    ###############################################
    #                                             #
    #           Likelihoods chirp masses events   #
    #                                             #
    ###############################################
    
    centerValuesLMc = np.zeros(len(names))

    #I want to do bootstrapping but the summing the error bars from
    #individual bootstrapping is not the same as the error bars from
    #bootstrapping over all events together.
    #dont want to keep recalculating the probabilities twice per event
    # Hence I store in array and sum correctly in post processing.
    #per event I need to store X numbers, where X is the number of
    #bootstrappings.
    LikelihoodsEvents = np.zeros(shape=(len(names),nrSamples))
    for nrB, observation in enumerate(names):
        #prepare the gaussians we are going to use for the observation
        #and the dx steps for integrating
        bayes.setObservation(observation=values[nrB], lowerError=lowerErrors[nrB], \
                             upperError=upperErrors[nrB], confidence=0.9)
        
        #What is our center value (i.e. with original rate)
        bayes.calculatePDFs(weights=rates)
        centerValuesLMc[nrB] = np.log10(bayes._PmodelGivenObservation )    
        #Now we have pre-bootstrapped the rates, so redo
        #last two lines per bootstrap and store in array
        for sample in range(nrSamples):
            #Reset rates according to Bootstrap sample
            s = boot.ReadSample(nrSample=sample, maskDCO=maskDCO)
            rateSample = np.multiply(s, rates)
            #recalculate the PDFs and store likelyhood
            bayes.calculatePDFs2(weights=rateSample)
            LikelihoodsEvents[nrB][sample] = np.log10(bayes._PmodelGivenObservation)
    #######################################################
    #        Calculating  Poisson statistic               #
    #######################################################
    
    surveyDurationsYr = {'O1':48./365.25, 'O2':118./365.25}
    Surveys            = np.unique(surveys)
    centerValuesLR    = np.zeros(len(Surveys))
    LikelihoodsRates  = np.zeros(shape=(len(Surveys),nrSamples))
    
    for nrs, survey in enumerate(np.unique(surveys)):
        #which events are in survey
        mask          = (surveys == survey)
        durationYr    = surveyDurationsYr[survey]
        nrObservations= np.sum(mask)
        
        rateCOMPASduringSurvey = np.sum(rates) * durationYr
        centerValuesLR[nrs]    = np.log10(bayes.poisson(rateCOMPASduringSurvey, nrObservations))
        for sample in range(nrSamples):
            #Reset rates according to Bootstrap sample
            s                        = boot.ReadSample(nrSample=sample, maskDCO=maskDCO)
            rateSample               = np.multiply(s, rates)
            rateCOMPASduringSurvey   = np.sum(rateSample) * durationYr
            LikelihoodsRates[nrs][sample] = np.log10(bayes.poisson(rateCOMPASduringSurvey, nrObservations))        
    
    ########################################################
    #      Combining likleyhoods                           #
    ########################################################
    #there must be a prettier way but no time nor brain for it now apologies
    
    #what I want is  McO1-+ RO1-+ McO2-+ RO2-+ Ltot-+ 5*3 entries
    results = np.zeros(15)
    
    #If you sort array at what index do you find lower and upper confidence 90%
    lower     = int((nrSamples/100.)*5)
    upper     = int((nrSamples/100.)*95)
    
    #reduce, sum,  and sort - center values is error
    maskO1      = (surveys == 'O1')
    centerMcO1  = np.sum(centerValuesLMc[maskO1])
    errorMcO1   = np.sort(np.sum(LikelihoodsEvents[maskO1], axis=0))- centerMcO1
    
    
    centerRO1   = centerValuesLR[0]
    errorRO1    = np.sort(LikelihoodsRates[0])- centerRO1
    
    results[0]  = centerMcO1
    results[1]  = errorMcO1[lower]
    results[2]  = errorMcO1[upper]
    results[3]  = centerRO1
    results[4]  = errorRO1[lower]
    results[5]  = errorRO1[upper]
 
    #reduce, sum,  and sort - center values is error
    maskO2      = (surveys == 'O2')
    centerMcO2  = np.sum(centerValuesLMc[maskO2])
    errorMcO2   = np.sort(np.sum(LikelihoodsEvents[maskO2], axis=0))- centerMcO2
    centerRO2   = centerValuesLR[1]
    errorRO2    = np.sort(LikelihoodsRates[1])- centerRO2
    
    results[6]  = centerMcO2
    results[7]  = errorMcO2[lower]
    results[8]  = errorMcO2[upper]
    results[9]  = centerRO2
    results[10] = errorRO2[lower]
    results[11] = errorRO2[upper]
    
    #total is sum of all likelhoods
    centerValue = np.sum(centerValuesLMc) + np.sum(centerValuesLR)
    LtotMc      = np.sum(LikelihoodsEvents, axis=0)
    LtotR       = np.sum(LikelihoodsRates, axis=0)
    LtotError   = np.sort(LtotMc+LtotR) - centerValue

    results[12] = centerValue
    results[13] = LtotError[lower]
    results[14] = LtotError[upper]
    return results

# Creating the Tables 

In [7]:
redoCalculation = False

if redoCalculation:
    #18 variations 15 entries
    results          = np.zeros(shape=(18,15))
    for Pessimistic in [False, True]:
        counterRow = 0
        for nrv, v in enumerate(a[b]):
            print h5file[v]['SFR_MZ_GSMF'][...]
            counterColumn   = 0
                 
            if Pessimistic:
                maskPessimistic = h5file[v]['BBH']['Pessimistic'][...].squeeze() == True
            else:
                #want them all
                maskPessimistic = np.ones(len(h5file[v]['BBH']['Pessimistic'][...].squeeze()), dtype=bool)
            chirpMasses   =  h5file[v]['BBH']['chirpMass'][...].squeeze()[maskPessimistic]
            rates         =  h5file[v]['BBH']['ObservedRate'][...].squeeze()[maskPessimistic]
            seedsInterest =  h5file[v]['BBH']['randomSeed'][...].squeeze()[maskPessimistic]
            
            #recover maskDCO for Bootstrap   
            originalDCOfile = h5.File(pathCOMPASOutput+'COMPASOutput.h5')
            seedsOriginal =  originalDCOfile['doubleCompactObjects']['seed'][...].squeeze()
            maskDCO       =  np.in1d(seedsOriginal, seedsInterest)

            results[nrv]   =  Likelihoods(chirpMasses, rates, maskDCO)
            print results[nrv]
        if Pessimistic:
            np.savetxt('LikelihoodsPessimistic.txt', results)
        else:
            np.savetxt('LikelihoodsOptimistic.txt', results)
        print 'Done'

In [8]:
o  = np.loadtxt('LikelihoodsOptimistic.txt')
p = np.loadtxt('LikelihoodsPessimistic.txt')

starter     = ['Madau et al. & Ma et al. (2004) & 1 &',' & & 2 &',' & & 3 &',
               ' & Langer et al.  &1 &',\
               ' &  &2 &',\
               ' &  &3 &',\
               ' & Langer et al., offset  &1 &',\
               ' &  &2 &',\
               ' &  &3 &',\
               'Strolger et al. & Ma et al. (2004) & 1 &',\
               ' & &2 &',\
               ' & &3 &',\
               ' & Langer et al. &1 &',\
               ' &  &2 &',\
               ' &  &3 &',\
               ' & Langer et al., offset &1 &',\
               ' &  &2 &',\
               ' &  &3 &',\
              ]
print r'\begin{table*}'
print r'\begin{tabular}{|l |l | l| lllll}'
print r'\multicolumn{3}{|c|}{Variation MSSFR} & \multicolumn{5}{|c|}{ Likelihoods}  \\'
print r'SFR & GSMF & MZ & $\rm \mathcal{L}_{Mc, O1}$ &   $\rm \mathcal{L}_{R, O1}$  &$\rm \mathcal{L}_{Mc, O2}$ &   $\rm \mathcal{L}_{R, O2}$ & $\rm \mathcal{L}_{tot}$ \\ \hline \hline'
print r' &&&&&&&\\'
print r'\multicolumn{8}{|c|}{Pessimistic} \\'
print r' &&&&&&&\\'


LMc   = 

for nrV in range(len(h5file.keys())):
    string = starter[nrV]
    row    = p[nrV]
    row = np.round(row, 2)
    string+= r'$ %s_{%s}^{+%s} $ & $ %s_{%s}^{+%s} $ & $ %s_{%s}^{+%s} $ & $ %s_{%s}^{+%s} $ & $ %s_{%s}^{+%s}$  \\'\
    %(row[0],row[1], row[2],row[3],row[4],row[5],row[6],row[7],row[8],\
      row[9],row[10],row[11],row[12],row[13],row[14])
        
    if nrV in [2,5,11,14]:
        string += '\cline{2-8}'
    if nrV == 8:
        string += '\hline'
    print string

print r'\hline '
print r' &&&&&&&\\'
print r'\multicolumn{8}{|c|}{Optimistic} \\'    
print r' &&&&&&&\\'

for nrV in range(len(h5file.keys())):
    string = starter[nrV]
    row    = o[nrV]
    row = np.round(row, 2)
    string+= r'$ %s_{%s}^{+%s} $ & $ %s_{%s}^{+%s} $ & $ %s_{%s}^{+%s} $ & $ %s_{%s}^{+%s} $ & $ %s_{%s}^{+%s}$  \\'\
    %(row[0],row[1], row[2],row[3],row[4],row[5],row[6],row[7],row[8],\
      row[9],row[10],row[11],row[12],row[13],row[14])
        
    if nrV in [2,5,11,14]:
        string += '\cline{2-8}'
    if nrV == 8:
        string += '\hline'
    print string

print r'\end{tabular}'
print r'\end{table*}'

\begin{table*}
\begin{tabular}{|l |l | l| lllll}
\multicolumn{3}{|c|}{Variation MSSFR} & \multicolumn{5}{|c|}{ Likelihoods}  \\
SFR & GSMF & MZ & $\rm \mathcal{L}_{Mc, O1}$ &   $\rm \mathcal{L}_{R, O1}$  &$\rm \mathcal{L}_{Mc, O2}$ &   $\rm \mathcal{L}_{R, O2}$ & $\rm \mathcal{L}_{tot}$ \\ \hline \hline
 &&&&&&&\\
\multicolumn{8}{|c|}{Pessimistic} \\
 &&&&&&&\\
Madau et al. & Ma et al. (2004) & 1 &$ -9.72_{-0.08}^{+0.08} $ & $ -0.97_{-0.02}^{+0.01} $ & $ -24.18_{-0.13}^{+0.12} $ & $ 0.0_{0.0}^{+0.0} $ & $ -34.87_{-0.16}^{+0.14}$  \\
 & & 2 &$ -9.87_{-0.05}^{+0.05} $ & $ -8.86_{-0.21}^{+0.21} $ & $ -22.55_{-0.07}^{+0.05} $ & $ 0.0_{0.0}^{+0.0} $ & $ -41.28_{-0.26}^{+0.24}$  \\
 & & 3 &$ -9.94_{-0.05}^{+0.04} $ & $ -11.9_{-0.18}^{+0.19} $ & $ -22.54_{-0.06}^{+0.05} $ & $ 0.0_{0.0}^{+0.0} $ & $ -44.38_{-0.22}^{+0.23}$  \\\cline{2-8}
 & Langer et al.  &1 &$ -9.77_{-0.03}^{+0.03} $ & $ -34.85_{-0.47}^{+0.5} $ & $ -22.47_{-0.05}^{+0.04} $ & $ 0.0_{0.0}^{+0.0} $ & $ -67.09_{-0.47}^{+0.49}$  \